# ASUM-DM Methodology Analysis - Case 1: Clustering

## 1. Etapas de entendimiento

### 1.1 Comprensión del negocio
**Objetivo:** Definir el problema, los objetivos de negocio y los requisitos de la solución.

**Problema:** Una empresa de retail busca implementar estrategias de marketing diferenciadas segmentando a sus clientes según su comportamiento.

**Objetivo de Negocio:** Confirmar o rechazar la hipótesis del equipo de mercadeo de que los clientes se pueden dividir en **3 grupos**.

### 1.2 Enfoque analítico
**Tipo de Problema:** Aprendizaje no supervisado (Clustering).

**Método de Construcción:** Se utilizarán y compararán múltiples algoritmos de agrupamiento divididos en 4 secciones:
1.  **Métodos de Partición:** K-Means y K-Medoids.
2.  **Métodos Jerárquicos:** Agglomerative Clustering (Dendrogramas).
3.  **Métodos Basados en Densidad:** DBSCAN (Optimización de Epsilon).
4.  **Métodos Basados en Distribución:** Gaussian Mixture Models (GMM).


## 2. Etapas de preparación

### 2.1 Requisitos de datos
Se requieren datos que describan el perfil demográfico y el comportamiento de compra.

**Diccionario de Datos:**
- `ID`: Identificador del cliente
- `Year_Birth`: Año de nacimiento
- `Education`: Nivel de educación
- `Marital_Status`: Estado civil
- `Income`: Ingresos anuales
- `Kidhome`: Cantidad de niños pequeños
- `Teenhome`: Cantidad de jóvenes
- `Dt_Customer`: Fecha de primera compra
- `Recency`: Días desde la última compra
- `MntWines`: Monto en vinos
- `MntFruits`: Monto en frutas
- `MntMeatProducts`: Monto en carnes
- `MntFishProducts`: Monto en pescados
- `MntSweetProducts`: Monto en dulces
- `MntGoldProds`: Monto en oro
- `NumDealsPurchases`: Compras con descuento
- `NumWebPurchases`: Compras web
- `NumCatalogPurchases`: Compras catálogo
- `NumStorePurchases`: Compras en tienda
- `NumWebVisitsMonth`: Visitas web mensuales
- `AcceptedCmp1-5`: Aceptación de campañas
- `Response`: Aceptación última campaña
- `Complain`: Quejas
- `Z_CostContact`: Costo de contacto
- `Z_Revenue`: Utilidad post-campaña


### 2.2 Recopilación de datos
Los datos se cargan desde `data/datos_caso_1.csv`.


In [14]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb

# Ensure utils can be imported
sys.path.append(os.path.abspath('..'))

from utils.entrega1.data_loader import load_data
from utils.entrega1.eda import get_basic_stats, check_missing_values_viz, plot_distributions_numerical, plot_boxplots, plot_pie_categorical, plot_correlation_matrix
from utils.entrega1.preprocessing import preprocess_pipeline, scale_features
from utils.entrega1.modeling import (run_clustering_models, compare_models, evaluate_clusters_kmeans, 
                                     visualize_clusters_pca, interpret_clusters, run_dbscan, 
                                     plot_dendrogram, plot_knn_distance, optimize_dbscan_grid)
from sklearn.cluster import AgglomerativeClustering, DBSCAN

# Load Data
df = load_data('../data/datos_caso_1.csv')
df.info()

ImportError: cannot import name 'plot_dendrogram' from 'utils.entrega1.modeling' (/Users/nicog/Desktop/compcient/utils/entrega1/modeling.py)

### 2.3 Comprensión de datos
Análisis Exploratorio de Datos (EDA).


In [ ]:
# Basic Statistics (Transposed)
display(get_basic_stats(df).T)

In [ ]:
# Missing Values Visualization
check_missing_values_viz(df)

In [ ]:
# Numerical Distributions
numerical_cols = ['Year_Birth', 'Income', 'Recency', 'MntWines', 'MntMeatProducts', 'NumWebVisitsMonth']
plot_distributions_numerical(df, numerical_cols)

In [ ]:
# Boxplots for Outliers
plot_boxplots(df, numerical_cols)

In [ ]:
# Categorical Distributions (Pie Charts)
categorical_cols = ['Education', 'Marital_Status']
plot_pie_categorical(df, categorical_cols)

In [ ]:
# Correlation Matrix
plot_correlation_matrix(df)

## 3. Etapas de ejecución

### 3.1 Preparación de los datos


In [ ]:
# Preprocessing Pipeline
df_prep = preprocess_pipeline(df)

# Select features for clustering
features_to_cluster = ['Age', 'Income', 'Kidhome', 'Teenhome', 'Recency', 
                       'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 
                       'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 
                       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 
                       'NumWebVisitsMonth', 'Tenure_Days', 'Total_Mnt', 'Total_Num_Purchases', 
                       'Family_Size']

features_to_cluster = [c for c in features_to_cluster if c in df_prep.columns]

# Scale Features
df_scaled, scaler = scale_features(df_prep, features_to_cluster)


### 3.2 Modelado Detallado
Evaluación profunda de 4 tipos de algoritmos.


#### 3.2.1 Métodos de Partición (K-Means)
Evaluación del número óptimo de clusters usando el método del codo y silueta.


In [ ]:
# K-Means Deep Dive
evaluate_clusters_kmeans(df_scaled, range(2, 9))

#### 3.2.2 Métodos Jerárquicos Aglomerativos
Análisis mediante Dendrogramas para visualizar la estructura de agrupamiento.


In [ ]:
# Average Linkage Dendrogram
plt.title('Hierarchical Clustering Dendrogram (Average)')
model_avg = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage='average')
model_avg = model_avg.fit(df_scaled)
plot_dendrogram(model_avg, truncate_mode='level', p=5)
plt.show()

# Ward Linkage Dendrogram
plt.title('Hierarchical Clustering Dendrogram (Ward)')
model_ward = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage='ward')
model_ward = model_ward.fit(df_scaled)
plot_dendrogram(model_ward, truncate_mode='level', p=5)
plt.axhline(y=30, linestyle='dashed', color='black') # Example cut
plt.show()

#### 3.2.3 Métodos Basados en Densidad (DBSCAN)
Identificación de clusters densos y ruido. Estimación de `epsilon` usando KNN y optimización de silueta.


In [ ]:
# KNN Distance Plot to estimate Epsilon
plot_knn_distance(df_scaled, k=5)

In [ ]:
# DBSCAN Grid Search for Optimal Hyperparameters
eps_range = np.arange(1.0, 3.0, 0.2)
min_samples_range = range(5, 25, 5)
print("Optimizing DBSCAN...")
dbscan_results = optimize_dbscan_grid(df_scaled, eps_range, min_samples_range)

#### 3.2.4 Métodos Basados en Modelos (GMM)
(Incluido en la comparación final)


### 3.3 Comparación Total de Modelos
Se comparan las métricas de Silueta y Davies-Bouldin de los mejores modelos encontrados.


In [ ]:
# Collect all results
results = run_clustering_models(df_scaled, n_clusters=3)

# Add optimized DBSCAN (Example based on heatmaps likely best)
# Assuming eps=1.8 works well from inspection, or pick best from dbscan_results
best_dbscan_row = dbscan_results.loc[dbscan_results['Score'].idxmax()]
print(f"Best DBSCAN params: Eps={best_dbscan_row['Epsilon']}, MinSamples={best_dbscan_row['MinSamples']}")

results['DBSCAN'] = run_dbscan(df_scaled, eps=best_dbscan_row['Epsilon'], min_samples=int(best_dbscan_row['MinSamples']))

# Review Agglomerative with Ward (k=3)
agg_ward = AgglomerativeClustering(n_clusters=3, linkage='ward')
results['Agglomerative-Ward'] = agg_ward.fit_predict(df_scaled)

# Compare
metrics_final = compare_models(df_scaled, results)
display(metrics_final)

### 3.4 Evaluación Final e Interpretación
Se seleccionan los clusters de **K-Means (k=3)** (o el mejor modelo) para confirmar la hipótesis y perfilar los grupos.


In [ ]:
# Visualize
visualize_clusters_pca(df_scaled, results['KMeans'], title='Final Clusters (K-Means k=3)')

# Interpretation
print("Perfilamiento de los 3 grupos:")
display(interpret_clusters(df_prep, results['KMeans'])[features_to_cluster].T)

## 4. Conclusiones
- **Validación de Hipótesis:** [Completar tras análisis: ¿Se sostienen los 3 grupos?]
- **Mejor Modelo:** [Completar tras ver gráfica de barras]
- **Caracterización:**
    - **Cluster 0:** ...
    - **Cluster 1:** ...
    - **Cluster 2:** ...
